# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
mens = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
mens['League'] = 'M'
mens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,117748.000000,117748.000000,117748.00000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,...,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000
mean,2014.146355,70.294986,1288.25451,75.878936,1283.138830,63.888287,0.068689,26.401824,55.760242,7.347445,...,20.159790,12.073403,17.732454,10.461740,21.625650,11.409722,13.888907,6.004739,2.868185,19.305780
std,6.515929,35.772556,105.34750,10.998547,104.795432,10.848767,0.305098,4.680314,7.456374,3.119260,...,6.068136,5.344049,7.081056,4.221039,4.518197,3.724567,4.382700,2.745969,2.019050,4.553353
min,2003.000000,0.000000,1101.00000,34.000000,1101.000000,20.000000,0.000000,10.000000,26.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,2009.000000,40.000000,1199.00000,68.000000,1192.000000,57.000000,0.000000,23.000000,51.000000,5.000000,...,16.000000,8.000000,13.000000,7.000000,19.000000,9.000000,11.000000,4.000000,1.000000,16.000000
50%,2014.000000,73.000000,1287.00000,75.000000,1282.000000,64.000000,0.000000,26.000000,55.000000,7.000000,...,20.000000,12.000000,17.000000,10.000000,21.000000,11.000000,14.000000,6.000000,3.000000,19.000000
75%,2020.000000,101.000000,1381.00000,83.000000,1374.000000,71.000000,0.000000,29.000000,60.000000,9.000000,...,24.000000,15.000000,22.000000,13.000000,25.000000,14.000000,17.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,1480.00000,149.000000,1480.000000,144.000000,6.000000,57.000000,103.000000,26.000000,...,59.000000,48.000000,65.000000,36.000000,49.000000,31.000000,41.000000,22.000000,18.000000,45.000000


In [3]:
womens = pd.read_csv('data/WRegularSeasonDetailedResults.csv')
womens['League'] = 'W'
womens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,...,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000
mean,2017.404609,69.183626,3285.116823,71.706633,3286.689554,57.242044,0.051708,25.847034,58.966574,6.276077,...,17.918413,10.507392,15.503808,11.360020,22.422122,10.935852,17.150745,7.109977,2.820839,18.192990
std,4.582659,36.157922,104.073477,11.536993,105.505327,10.960867,0.259072,4.978157,7.969144,3.127369,...,6.456006,4.936838,6.630184,4.640191,4.936106,3.805935,5.277180,3.192300,2.062848,4.556919
min,2010.000000,0.000000,3101.000000,30.000000,3101.000000,11.000000,0.000000,9.000000,30.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,3.000000
25%,2013.000000,36.000000,3196.000000,64.000000,3195.000000,50.000000,0.000000,22.000000,53.000000,4.000000,...,13.000000,7.000000,11.000000,8.000000,19.000000,8.000000,13.000000,5.000000,1.000000,15.000000
50%,2017.000000,73.000000,3283.000000,71.000000,3287.000000,57.000000,0.000000,25.000000,59.000000,6.000000,...,17.000000,10.000000,15.000000,11.000000,22.000000,11.000000,17.000000,7.000000,2.000000,18.000000
75%,2022.000000,101.000000,3376.000000,79.000000,3377.000000,64.000000,0.000000,29.000000,64.000000,8.000000,...,22.000000,14.000000,20.000000,14.000000,26.000000,13.000000,20.000000,9.000000,4.000000,21.000000
max,2025.000000,132.000000,3480.000000,140.000000,3480.000000,130.000000,5.000000,58.000000,113.000000,30.000000,...,80.000000,37.000000,52.000000,38.000000,53.000000,34.000000,49.000000,26.000000,21.000000,47.000000


The IDs are definitely distinct so we can combine into a single dataframe

In [4]:
data = pd.concat([mens, womens])

Get the distinct team/Years

In [5]:
teams = pd.concat([data[['WTeamID', 'Season', 'League']].rename(columns={'WTeamID': 'TeamID'}),
                   data[['LTeamID', 'Season', 'League']].rename(columns={'LTeamID': 'TeamID'})]).drop_duplicates().reset_index()

Define the training data. The x's will be the indexes of two team IDs and their average stats, the y's will be the score difference.

In [6]:
teamMapping = {(x.TeamID, x.Season): x.Index for x in teams.itertuples()}

In [7]:
winningStats = data[['WTeamID', 'Season', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
                    'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
winningStats.columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 
                      'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
losingStats = data[['LTeamID', 'Season', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 
                             'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
losingStats.columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 
                     'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
teamStats = pd.concat([winningStats, losingStats]).groupby(['TeamID', 'Season']).mean()

In [8]:
def gen_dataset(data):
    winners = data.apply(lambda x: teamMapping[(x.WTeamID, x.Season)], axis=1)
    losers = data.apply(lambda x: teamMapping[(x.LTeamID, x.Season)], axis=1)
    winner_stats = data.apply(lambda x: teamStats.loc[(x.WTeamID, x.Season)], axis=1)
    loser_stats = data.apply(lambda x: teamStats.loc[(x.LTeamID, x.Season)], axis=1)
    winning_matchups = np.concatenate([np.stack([winners, losers], axis=1), winner_stats, loser_stats], axis=1)
    losing_matchups = np.concatenate([np.stack([losers, winners], axis=1), loser_stats, winner_stats], axis=1)
    x_tensor = torch.from_numpy(np.concatenate([winning_matchups, losing_matchups])).double()
    y_tensor = torch.from_numpy(np.concatenate([(data.WScore-data.LScore), (data.LScore-data.WScore)]).reshape((-1,1))).double()
    return TensorDataset(x_tensor, y_tensor)

In [9]:
fname = 'dataset.pt'
if os.path.isfile(fname):
    dataset=torch.load(fname, weights_only=False)
else:
    dataset = gen_dataset(data)
    torch.save(dataset, fname)

Generate the train/validation split

In [10]:
batch_size=200

generator = torch.Generator().manual_seed(20250217)
train_data, validation_data = torch.utils.data.random_split(dataset, [0.95, 0.05], generator=generator)
train_loader = DataLoader(train_data, batch_size=batch_size)
validation_loader = DataLoader(validation_data, batch_size=batch_size)

## The Model
Define the model. Combine the embeddings for the two teams, go to a hidden layer, and then output to a prediction if the first team won

In [11]:
class Model(nn.Module):
    def __init__(self, embedding_size, model_sizes, dropout, stats_size=14):
        super(Model, self).__init__()
        hid1, hid2 = model_sizes
        self.embedding = nn.Embedding(len(teams), embedding_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(2*embedding_size+2*stats_size, hid1)
        self.fc2 = nn.Linear(hid1, hid2)
        self.score_fc = nn.Linear(hid2, 1)
        self.result_fc = nn.Linear(hid2, 1)
        self.double()

    def forward(self, x):
        team = self.embedding(x[:,0].int())
        opponent = self.embedding(x[:,1].int())
        matchup = self.dropout1(torch.cat([team, opponent, x[:,2:]], axis=1))
        hidden1 = self.dropout2(F.relu(self.fc1(matchup)))
        hidden2 = self.dropout3(F.relu(self.fc2(hidden1)))
        score = self.score_fc(hidden2)
        result = F.sigmoid(self.result_fc(hidden2))
        return score, result
        

In [12]:
model = Model(embedding_size=512, model_sizes=(64,64), dropout=0.25).to(device)

## Training the model

Define the training function

In [13]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

def train(data, model, loss_fn, optimizer, full_loss=True):
    size = len(data.dataset)
    model.train()
    for batch, (x, y) in enumerate(data):
        x = x.to(device)
        y = y.to(device)
        pred_score, pred_result = model(x)
        actual_result = (y > 0).double()
        score_loss = loss_fn(pred_score, y)
        result_loss = loss_fn(pred_result, actual_result)
        if full_loss:
            (score_loss + 10 * result_loss).backward()
        else:
            result_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            score_loss, result_loss, current = score_loss.item(), result_loss.item(), (batch + 1) * len(x)
            print(f"score loss: {score_loss:>7f}, result loss: {result_loss:>7f} [{current:>6d}/{size:>6d}]", end="\r")

Define the testing function

In [14]:
def test(data, model, loss_fn, label="Test"):
    size = len(data.dataset)
    num_batches = len(data)
    model.eval()
    score_loss, result_loss, correct = 0, 0, 0
    with torch.no_grad():
        for x, y in data:
            x = x.to(device)
            y = y.to(device)
            score_pred, result_pred = model(x)
            actual_result = (y > 0).double()
            score_loss += loss_fn(score_pred, y).item()
            result_loss += loss_fn(result_pred, actual_result).item()
            correct += ((result_pred >= 0.5) == (actual_result == 1)).type(torch.float).sum().item()
    score_loss /= num_batches
    result_loss /= num_batches
    correct /= size
    print(f"{label}: Accuracy: {(100*correct):>0.1f}%, Score loss: {score_loss:>8f}, Result loss: {result_loss:>8f}")

Train the model

In [15]:
n_epochs = 20
for i in range(n_epochs):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train: Accuracy: 68.1%, Score loss: 222.305757, Result loss: 0.217611
Validation: Accuracy: 67.4%, Score loss: 223.419624, Result loss: 0.219579
Epoch 1
Train: Accuracy: 71.0%, Score loss: 199.977312, Result loss: 0.202261
Validation: Accuracy: 69.7%, Score loss: 203.794183, Result loss: 0.206275
Epoch 2
Train: Accuracy: 73.1%, Score loss: 177.788741, Result loss: 0.187606
Validation: Accuracy: 71.6%, Score loss: 184.024408, Result loss: 0.193375
Epoch 3
Train: Accuracy: 74.5%, Score loss: 162.748506, Result loss: 0.177572
Validation: Accuracy: 72.9%, Score loss: 170.573268, Result loss: 0.184614
Epoch 4
Train: Accuracy: 75.2%, Score loss: 155.720576, Result loss: 0.173916
Validation: Accuracy: 73.6%, Score loss: 163.801913, Result loss: 0.181086
Epoch 5
Train: Accuracy: 75.7%, Score loss: 141.599888, Result loss: 0.166620
Validation: Accuracy: 73.9%, Score loss: 150.712891, Result loss: 0.174690
Epoch 6
Train: Accuracy: 76.0%, Score loss: 137.986868, Result loss: 0.164618
Vali

Fine tune with only the result

In [16]:
for i in range(5):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer, full_loss=False)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train: Accuracy: 76.6%, Score loss: 116.458346, Result loss: 0.156648
Validation: Accuracy: 75.1%, Score loss: 127.821253, Result loss: 0.166302
Epoch 1
Train: Accuracy: 76.6%, Score loss: 116.414394, Result loss: 0.156487
Validation: Accuracy: 75.1%, Score loss: 127.756559, Result loss: 0.166259
Epoch 2
Train: Accuracy: 76.7%, Score loss: 116.456290, Result loss: 0.156374
Validation: Accuracy: 75.1%, Score loss: 127.752706, Result loss: 0.166230
Epoch 3
Train: Accuracy: 76.7%, Score loss: 116.575089, Result loss: 0.156193
Validation: Accuracy: 75.2%, Score loss: 127.767654, Result loss: 0.166204
Epoch 4
Train: Accuracy: 76.8%, Score loss: 117.283729, Result loss: 0.155798
Validation: Accuracy: 75.2%, Score loss: 128.271922, Result loss: 0.166076


With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [17]:
mens_tourney = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
womens_tourney = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
tourney = pd.concat([mens_tourney, womens_tourney])

tourney_dataset = gen_dataset(tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=batch_size)

In [18]:
test(tourney_loader, model, loss_fn, label="Tourney")

Tourney: Accuracy: 73.7%, Score loss: 138.969193, Result loss: 0.170096


When it comes to tournament results we get about 7 out of 10 results. The lower result is likely due to teams having increased pairity.

### Performance by year


In [19]:
for season in tourney.Season.unique():
    loader = DataLoader(gen_dataset(tourney[tourney.Season == season]), batch_size=batch_size)
    test(loader, model, loss_fn, label=f"{season} Tournament")

2003 Tournament: Accuracy: 68.0%, Score loss: 121.611516, Result loss: 0.189804
2004 Tournament: Accuracy: 70.3%, Score loss: 115.757377, Result loss: 0.182849
2005 Tournament: Accuracy: 76.6%, Score loss: 89.355132, Result loss: 0.169172
2006 Tournament: Accuracy: 66.4%, Score loss: 101.377029, Result loss: 0.200347
2007 Tournament: Accuracy: 78.1%, Score loss: 104.546692, Result loss: 0.154620
2008 Tournament: Accuracy: 81.2%, Score loss: 143.711519, Result loss: 0.156952
2009 Tournament: Accuracy: 72.7%, Score loss: 138.729526, Result loss: 0.168544
2010 Tournament: Accuracy: 73.2%, Score loss: 131.186752, Result loss: 0.168534
2011 Tournament: Accuracy: 73.8%, Score loss: 122.844959, Result loss: 0.157930
2012 Tournament: Accuracy: 76.2%, Score loss: 103.611298, Result loss: 0.141292
2013 Tournament: Accuracy: 73.8%, Score loss: 128.716313, Result loss: 0.162950
2014 Tournament: Accuracy: 71.5%, Score loss: 156.227938, Result loss: 0.153391
2015 Tournament: Accuracy: 81.9%, Score l

## Inspect the model
First what are the sizes of the smallest input and output weights

In [20]:
print(f"Embedding min: {model.embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"FC min: {model.result_fc.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")

Embedding min: 3.386301
FC min: 0.009798


Calculate the average gradient for each input feature

In [21]:
def feature_eval(model, data):
    model.eval()
    embedding_grads = torch.zeros(model.embedding.weight.shape[1]).to(device)
    stats_grads = torch.zeros(28).to(device)
    size = len(data.dataset)
    for batch, (x, y) in enumerate(data):
        x = x.to(device)
        y = y.to(device)
        x.requires_grad = True
        _, pred_result = model(x)
        embedding_grads += torch.autograd.grad(model(x)[1].mean(), model.embedding.parameters())[0].sum(axis=0)
        stats_grads += torch.autograd.grad(model(x)[1].mean(), x)[0].sum(axis=0)[2:]
    return embedding_grads/size, stats_grads

In [22]:
embedding_weights, stats_weights = feature_eval(model, train_loader)

In [23]:
embedding_weights.abs().sum()

tensor(0.0008, device='cuda:0')

In [24]:
for stat, weight in zip([f"W{stat}" for stat in teamStats] + [f"L{stat}" for stat in teamStats], stats_weights):
    print(f"{stat}:\t{weight:>4f}")

WScore:	2.430429
WFGM:	0.836126
WFGA:	0.044090
WFGM3:	0.921646
WFGA3:	-0.076395
WFTM:	0.626387
WFTA:	0.213781
WOR:	-0.054524
WDR:	0.124199
WAst:	0.899460
WTO:	-0.266915
WStl:	0.382566
WBlk:	3.989772
WPF:	-0.249774
LScore:	-1.850683
LFGM:	-0.862128
LFGA:	-0.019124
LFGM3:	-1.659680
LFGA3:	-0.121787
LFTM:	-0.491174
LFTA:	-0.418190
LOR:	-0.328272
LDR:	-0.163861
LAst:	-1.445212
LTO:	0.636673
LStl:	-0.525856
LBlk:	-4.166779
LPF:	0.259018


## Generating the submission file
### Phase 1

Write the results

In [25]:
with open('submission.csv', 'w') as f:
    f.write("ID,Pred\n")
    season=2025
    for league in ('M', 'W'):
        teams_to_test = sorted(teams[(teams.Season==season) & (teams.League==league)].TeamID.values)
        matchups = [(t1, t2) for t1 in teams_to_test for t2 in teams_to_test if t1 < t2]
        matchups_tensor = torch.Tensor(np.array(
            [np.concat([[teamMapping[(t1, season)], teamMapping[(t2, season)]],
                        teamStats.loc[(t1, season)].values,
                        teamStats.loc[(t2, season)].values])
             for (t1, t2) in matchups])).int().to(device)
        _, predictions = model(matchups_tensor)
        for (t1, t2), pred in zip(matchups, predictions):
            f.write(f"{season}_{t1.item()}_{t2.item()},{pred.item()}\n")

## Save the model

In [26]:
torch.save(model.state_dict(), 'model.pth')